<a href="https://colab.research.google.com/github/salim-hbk/ai-ml/blob/main/GraphBasedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Horse or Human? In-graph training**

loading batches

calculating gradients

updating parameters

calculating validation accuracy

repeating the loop until convergence

Before to all little intro of Graph based model


---


*   Normally tensorflow as 2 types of execution Graph based and eager execution
*   Graph based execution is faster than eager execution during computaion as literally everything in Graph. it executes everything completing computational graph tree
*   Graph functions are polimorphic in nature, if you decalre any argument in graph function it can take any data types at run time
*   Problem with graph based model is not easily debuggable and developer friendly. 






In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [5]:
a = tf.Variable(2.)
b = tf.Variable(1.)
@tf.function
def f(x, y):
  b.assign(y+b)
  a.assign_add(a*b)
  return a+b

print(f(1,2))
print(tf.autograph.to_code(f.python_function))
#ag = auto graph

tf.Tensor(11.0, shape=(), dtype=float32)
def tf__f(x, y):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        ag__.converted_call(ag__.ld(b).assign, ((ag__.ld(y) + ag__.ld(b)),), None, fscope)
        ag__.converted_call(ag__.ld(a).assign_add, ((ag__.ld(a) * ag__.ld(b)),), None, fscope)
        try:
            do_return = True
            retval_ = (ag__.ld(a) + ag__.ld(b))
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [8]:
splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'], data_dir='./data')

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Shuffling and writing examples to ./data/horses_or_humans/3.0.0.incompleteBYYQWS/horses_or_humans-train.tfrecord


Shuffling and writing examples to ./data/horses_or_humans/3.0.0.incompleteBYYQWS/horses_or_humans-test.tfrecord


Dataset horses_or_humans downloaded and prepared to ./data/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [9]:
BATCH_SIZE = 32
IMAGE_SIZE = 224

@tf.function
def map_fn(img, label):
  img = tf.image.resize(img, [IMAGE_SIZE, IMAGE_SIZE])
  img /=255.
  return img, label


Apply pre-processing to the datasets 

In [10]:
def prepare_dataset(train_examples, validation_examples, test_examples, num_examples, map_fn, batch_size):
  train_ds = train_examples.map(map_fn).shuffle(buffer_size = num_examples).batch(batch_size)
  valid_ds = validation_examples.map(map_fn).batch(batch_size)
  test_ds = test_examples.map(map_fn).batch(batch_size)
  return train_ds, valid_ds, test_ds

In [11]:
train_ds, valid_ds, test_ds = prepare_dataset(train_examples, validation_examples, test_examples, num_examples, map_fn, BATCH_SIZE)

**Define the model**
We will load existing model and modify it for our use